# Follow-up Yelp API Restaurant Calls For Coordinates > 1k Results

See calls_by_coords_rating.ipynb for detailed info on code logic

An additional search for these 2 coordinates is warranted in efforts to obtain this missing info. Yelp API allows for sorting of results before retrieving the first 1k results. Adding this supplemental data to the main data will ensure the inclusion of the top rated restaurants needed for this project while also giving us the maximum possible database of Yelp restaurants in the Bay under these parameters.

Write data to restaurant_data_coord.csv

In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from grid_coordinates.csv
- Build function that calls and writes restaurant data to new csv, given pair of coordinates
- Run list of coordinates through function

In [12]:
coords_df = pd.read_csv("Resources/grid_coordinates_over1k.csv", header=None)
lat = []
lng = []
for x in coords_df[0]:
    x = x.split(", ")
    lat.append(x[0])
    lng.append(x[1])
    
coords_df = pd.DataFrame({'Lat': lat, 'Lng': lng})
coords_df

,Lat,Lng
0,37.842211,-122.23913
1,37.842211,-122.29470800000001
2,37.320235,-121.905662
3,37.363733,-121.905662
4,37.842211,-122.405864
5,37.755215,-122.46144199999999
6,37.711717,-122.46144199999999
7,37.798713,-122.405864
8,37.755215,-122.23913


In [13]:
# Function returns up to 1000 restaurant listings for input coordinates
def get_restaurants(lat, lng, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls, radius 8046 meters = 5 miles
        params = {"term":"restaurants", "sort_by":"rating", "latitude":lat, "longitude":lng, "radius":4829, 
                  "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Log history
            count += 1
            print(f"Now processing query set {count} of max 20")
            
            # Breaks if no further entries in query
            if response["businesses"] == []:
                break
            
            else:
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        
                        # Replace both price & missing category data with empty string
                        if biz["categories"] == []:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", "", biz["id"]])
                        else:
                            restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], 
                                                    biz["coordinates"]["latitude"], 
                                                    biz["coordinates"]["longitude"], 
                                                    biz["location"]["address1"], 
                                                    biz["location"]["zip_code"], biz["rating"], 
                                                    biz["review_count"], "", biz["categories"][0]["title"], 
                                                    biz["id"]])

                    # Replace missing category data with empty string
                    elif biz["categories"] == []:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], "", biz["id"]])

                    else:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('Resources/restaurant_data_ratings.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()
    
    # Returns total count of restaurants in coordinates
    return response["total"]


In [14]:
# Track # of coordinates processed
count = 0

# List to track total restaurants found in coordinate
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of coordinates
lat_coords = [lat for lat in coords_df["Lat"]]
lng_coords = [lng for lng in coords_df["Lng"]]

for index in range(len(lat_coords)):
                   
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(lat_coords[index], lng_coords[index] , api_key))
    
    # Print log history
    count += 1
    rem = len(lat_coords) - count
    print("-----------------------------------------")
    if count == len(lat_coords):
        print("Full list of coordinates processed!")
    elif count == len(lat_coords) - 1:
        print("Now getting results for final coordinates. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for coordinates {count} complete")
        print(f"Getting results for next coordinates.. there are {rem} left")
        print("-----------------------------------------")

coords_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
Now processing query set 3 of max 20
Now processing query set 4 of max 20
Now processing query set 5 of max 20
Now processing query set 6 of max 20
Now processing query set 7 of max 20
Now processing query set 8 of max 20
Now processing query set 9 of max 20
Now processing query set 10 of max 20
Now processing query set 11 of max 20
Now processing query set 12 of max 20
Now processing query set 13 of max 20
Now processing query set 14 of max 20
Now processing query set 15 of max 20
Now processing query set 16 of max 20
Now processing query set 17 of max 20
Now processing query set 18 of max 20
Now processing query set 19 of max 20
Now processing query set 20 of max 20
-----------------------------------------
Data retrieval for coordinates 1 complete
Getting results for next coordinates.. there are 8 left
-----------------------------------------
Now processin

In [16]:
# Show coords_df with updated total # of restaurants
# Should still match previous results. ie 2700 & 2600 respectively
coords_df

,Lat,Lng,Total # of Restaurants
0,37.842211,-122.23913,1400
1,37.842211,-122.29470800000001,1400
2,37.320235,-121.905662,1100
3,37.363733,-121.905662,1100
4,37.842211,-122.405864,1400
5,37.755215,-122.46144199999999,2100
6,37.711717,-122.46144199999999,1200
7,37.798713,-122.405864,3200
8,37.755215,-122.23913,1000
